In [ ]:
!pip install tensorflow

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report


In [ ]:
from tensorflow.keras.layers import Conv2D, Activation, Multiply

# Create directories for training and testing data
os.makedirs("/content/drive/MyDrive/GEN-AI/training", exist_ok=True)
os.makedirs("/content/drive/MyDrive/GEN-AI/testing", exist_ok=True)

def spatial_attention(input_feature):
    # Compute attention map
    attention = Conv2D(1, (1, 1), padding='same', activation='sigmoid')(input_feature)
    # Apply attention to the input feature
    return Multiply()([input_feature, attention])

# Define the CNN model
model = Sequential()
model.add(Convolution2D(64, (5, 5), input_shape=(48, 48, 1), activation='relu'))
model.add(Convolution2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(spatial_attention(model.output))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=7, activation='softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 44, 44, 64)        1664      
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 64)        102464    
                                                                 
 max_pooling2d (MaxPooling2  (None, 20, 20, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 20, 20, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 18, 18, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 128)       147584    
                                                        

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define data generators for training and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=False)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/GEN-AI/training',
    target_size=(48, 48),
    batch_size=512,
    color_mode="grayscale",
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/GEN-AI/testing',
    target_size=(48, 48),
    batch_size=512,
    color_mode="grayscale",
    class_mode='categorical')

# Define callbacks for model saving
filepath = os.path.join("/content/drive/MyDrive/GEN-AI/Models/model_v{epoch}.hdf5")
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks = [checkpoint]


Found 13559 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [ ]:
# Train the model
nb_train_samples = 13559
nb_validation_samples = 7178
batch_size = 512

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks
)


<ipython-input-9-b69dd3f2e998>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/50
26/26 [==============================] - ETA: 0s - loss: 1.2928 - accuracy: 0.3321 
Epoch 1: val_accuracy improved from -inf to 0.21889, saving model to /content/drive/MyDrive/GEN-AI/Models/model_v1.hdf5
26/26 [==============================] - 1268s 50s/step - loss: 1.2928 - accuracy: 0.3321 - val_loss: 3.2535 - val_accuracy: 0.2189
Epoch 2/50


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


26/26 [==============================] - ETA: 0s - loss: 1.2089 - accuracy: 0.3642 
Epoch 2: val_accuracy improved from 0.21889 to 0.24456, saving model to /content/drive/MyDrive/GEN-AI/Models/model_v2.hdf5
26/26 [==============================] - 524s 20s/step - loss: 1.2089 - accuracy: 0.3642 - val_loss: 3.5081 - val_accuracy: 0.2446
Epoch 3/50
26/26 [==============================] - ETA: 0s - loss: 1.1970 - accuracy: 0.3825 
Epoch 3: val_accuracy improved from 0.24456 to 0.25293, saving model to /content/drive/MyDrive/GEN-AI/Models/model_v3.hdf5
26/26 [==============================] - 521s 20s/step - loss: 1.1970 - accuracy: 0.3825 - val_loss: 4.2471 - val_accuracy: 0.2529
Epoch 4/50
26/26 [==============================] - ETA: 0s - loss: 1.1829 - accuracy: 0.4042 
Epoch 4: val_accuracy did not improve from 0.25293
26/26 [==============================] - 524s 20s/step - loss: 1.1829 - accuracy: 0.4042 - val_loss: 4.3637 - val_accuracy: 0.2408
Epoch 5/50
26/26 [==================

In [ ]:
# Train the model
nb_train_samples = 13559
nb_validation_samples = 7178
batch_size = 512

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=callbacks
)


<ipython-input-15-c2abdaa9cc4a>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/30
26/26 [==============================] - ETA: 0s - loss: 1.1179 - accuracy: 0.4795 
Epoch 1: val_accuracy did not improve from 0.30259
26/26 [==============================] - 525s 20s/step - loss: 1.1179 - accuracy: 0.4795 - val_loss: 4.6095 - val_accuracy: 0.2984
Epoch 2/30
26/26 [==============================] - ETA: 0s - loss: 1.1023 - accuracy: 0.4863 
Epoch 2: val_accuracy improved from 0.30259 to 0.30301, saving model to /content/drive/MyDrive/GEN-AI/Models/model_v2.hdf5
26/26 [==============================] - 524s 20s/step - loss: 1.1023 - accuracy: 0.4863 - val_loss: 4.3910 - val_accuracy: 0.3030
Epoch 3/30
26/26 [==============================] - ETA: 0s - loss: 1.0843 - accuracy: 0.5024 
Epoch 3: val_accuracy improved from 0.30301 to 0.31627, saving model to /content/drive/MyDrive/GEN-AI/Models/model_v3.hdf5
26/26 [==============================] - 525s 20s/step - loss: 1.0843 - accuracy: 0.5024 - val_loss: 5.1845 - val_accuracy: 0.3163
Epoch 4/30
26/26 [=======